# 銘柄スクリーニング

このノートブックでは、条件に合った銘柄を見つける方法を学びます。

## スクリーニングとは？

スクリーニングは、特定の条件で銘柄をフィルタリングして、投資候補を絞り込むことです。
- 業種や市場で絞り込み
- 株価や出来高で条件設定
- テクニカル指標でフィルタリング

## 目次
1. 環境設定
2. 業種別スクリーニング
3. 株価条件でのスクリーニング
4. 出来高スクリーニング
5. 複合条件スクリーニング

## 1. 環境設定

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 認証情報の設定
os.environ['JQUANTS_MAIL_ADDRESS'] = 'your_email@example.com'
os.environ['JQUANTS_PASSWORD'] = 'your_password'

import pyjquants as pjq
print('準備完了')

## 2. 業種別スクリーニング

### 市場の業種一覧を確認

In [ ]:
# 市場オブジェクトを作成
market = pjq.Market()

# 33業種分類を取得
sectors = market.sectors_33

print('33業種分類:')
print('-' * 40)
for i, sector in enumerate(sectors, 1):
    print(f'{i:2}. {sector.code}: {sector.name}')

### 特定業種の銘柄を取得

In [ ]:
# サンプル銘柄リスト（実際のAPIでは全銘柄を取得可能）
sample_codes = [
    '7203', '7267', '7201',  # 自動車
    '6758', '6861', '6902',  # 電機
    '9984', '9433', '9434',  # 情報通信
    '8306', '8316', '8411',  # 銀行
    '4502', '4503', '4568',  # 医薬品
]

# 各銘柄の情報を取得
stocks_info = []
for code in sample_codes:
    try:
        stock = pjq.Stock(code)
        latest = stock.latest_price
        stocks_info.append({
            'code': code,
            'name': stock.name,
            'sector': stock.sector_33,
            'market': stock.market_segment.name,
            'price': float(latest.close),
            'volume': latest.volume,
        })
    except Exception as e:
        print(f'{code}: エラー - {e}')

df = pd.DataFrame(stocks_info)
print(f'取得銘柄数: {len(df)}')
df

In [ ]:
# 業種別に集計
sector_counts = df.groupby('sector').size().sort_values(ascending=False)

print('業種別銘柄数:')
print('-' * 30)
for sector, count in sector_counts.items():
    print(f'{sector}: {count}銘柄')

## 3. 株価条件でのスクリーニング

### 株価レンジでフィルタリング

In [ ]:
# 1,000円〜5,000円の銘柄
price_min = 1000
price_max = 5000

filtered = df[(df['price'] >= price_min) & (df['price'] <= price_max)]

print(f'株価 {price_min:,}円〜{price_max:,}円 の銘柄:')
print('-' * 50)
filtered[['code', 'name', 'sector', 'price']].sort_values('price')

### 値上がり銘柄を探す

In [ ]:
# 各銘柄の直近パフォーマンスを計算
performance = []

for code in sample_codes:
    try:
        stock = pjq.Stock(code)
        prices = stock.prices
        
        if len(prices) >= 5:
            # 5日間のリターン
            current = float(prices.iloc[-1]['close'])
            past = float(prices.iloc[-5]['close'])
            return_5d = (current - past) / past * 100
            
            performance.append({
                'code': code,
                'name': stock.name,
                'price': current,
                'return_5d': return_5d,
            })
    except Exception as e:
        pass

perf_df = pd.DataFrame(performance).sort_values('return_5d', ascending=False)

print('5日間リターン ランキング:')
print('-' * 50)
perf_df

## 4. 出来高スクリーニング

### 出来高急増銘柄

In [ ]:
# 出来高の変化を分析
volume_analysis = []

for code in sample_codes:
    try:
        stock = pjq.Stock(code)
        prices = stock.prices
        
        if len(prices) >= 10:
            # 直近出来高 vs 10日平均
            current_vol = prices.iloc[-1]['volume']
            avg_vol = prices['volume'].tail(10).mean()
            vol_ratio = current_vol / avg_vol
            
            volume_analysis.append({
                'code': code,
                'name': stock.name,
                'volume': current_vol,
                'avg_volume': avg_vol,
                'vol_ratio': vol_ratio,
            })
    except Exception as e:
        pass

vol_df = pd.DataFrame(volume_analysis).sort_values('vol_ratio', ascending=False)

print('出来高比率（直近/10日平均）ランキング:')
print('-' * 60)
vol_df

In [ ]:
# 出来高急増銘柄（平均の1.5倍以上）
high_volume = vol_df[vol_df['vol_ratio'] >= 1.5]

print('出来高急増銘柄（1.5倍以上）:')
if len(high_volume) > 0:
    for _, row in high_volume.iterrows():
        print(f"  {row['name']} ({row['code']}): {row['vol_ratio']:.2f}倍")
else:
    print('  該当銘柄なし')

## 5. 複合条件スクリーニング

### スクリーニング関数の作成

In [ ]:
def screen_stocks(codes, conditions):
    """
    複合条件でスクリーニング
    
    conditions: dict
        - price_min: 最低株価
        - price_max: 最高株価
        - volume_min: 最低出来高
        - return_5d_min: 5日リターン下限
        - sectors: 対象業種リスト
    """
    results = []
    
    for code in codes:
        try:
            stock = pjq.Stock(code)
            prices = stock.prices
            latest = stock.latest_price
            
            price = float(latest.close)
            volume = latest.volume
            sector = stock.sector_33
            
            # 5日リターン
            if len(prices) >= 5:
                return_5d = (price - float(prices.iloc[-5]['close'])) / float(prices.iloc[-5]['close']) * 100
            else:
                return_5d = 0
            
            # 条件チェック
            if 'price_min' in conditions and price < conditions['price_min']:
                continue
            if 'price_max' in conditions and price > conditions['price_max']:
                continue
            if 'volume_min' in conditions and volume < conditions['volume_min']:
                continue
            if 'return_5d_min' in conditions and return_5d < conditions['return_5d_min']:
                continue
            if 'sectors' in conditions and sector not in conditions['sectors']:
                continue
            
            results.append({
                'code': code,
                'name': stock.name,
                'sector': sector,
                'price': price,
                'volume': volume,
                'return_5d': return_5d,
            })
            
        except Exception as e:
            pass
    
    return pd.DataFrame(results)

print('スクリーニング関数を定義しました')

In [ ]:
# スクリーニング実行例
conditions = {
    'price_min': 1000,      # 1,000円以上
    'price_max': 10000,     # 10,000円以下
    'volume_min': 100000,   # 出来高10万株以上
}

screened = screen_stocks(sample_codes, conditions)

print('スクリーニング条件:')
print(f'  株価: {conditions["price_min"]:,}円 〜 {conditions["price_max"]:,}円')
print(f'  出来高: {conditions["volume_min"]:,}株以上')
print()
print(f'該当銘柄: {len(screened)}件')
print('-' * 60)
screened.sort_values('return_5d', ascending=False)

### スクリーニング結果の可視化

In [ ]:
if len(screened) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 株価分布
    axes[0].bar(screened['name'], screened['price'], color='steelblue')
    axes[0].set_title('Stock Prices', fontsize=12)
    axes[0].set_ylabel('Price (JPY)')
    axes[0].tick_params(axis='x', rotation=45)
    
    # 5日リターン
    colors = ['green' if x >= 0 else 'red' for x in screened['return_5d']]
    axes[1].bar(screened['name'], screened['return_5d'], color=colors)
    axes[1].set_title('5-Day Return (%)', fontsize=12)
    axes[1].set_ylabel('Return (%)')
    axes[1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.show()
else:
    print('該当銘柄がありません')

## まとめ

このノートブックでは以下を学びました：

- **業種別スクリーニング**: 33業種分類での絞り込み
- **株価条件**: 価格レンジでのフィルタリング
- **出来高分析**: 出来高急増銘柄の検出
- **複合条件**: 複数条件の組み合わせ

### 応用アイデア

- RSIやMACDなどのテクニカル指標を追加
- PERやPBRなどのファンダメンタル指標でフィルタリング
- 定期的なスクリーニングの自動化

---

**免責事項**: このノートブックは教育目的であり、投資助言ではありません。